#### Native App Provider Setup

In [ ]:
--Create NASPCS role and Grant Privileges
use role accountadmin;
create role if not exists naspcs_role;
grant role naspcs_role to role accountadmin;
grant create integration on account to role naspcs_role;
grant create compute pool on account to role naspcs_role;
grant create warehouse on account to role naspcs_role;
grant create database on account to role naspcs_role;
grant create application package on account to role naspcs_role;
grant create application on account to role naspcs_role with grant option;
grant bind service endpoint on account to role naspcs_role;


In [ ]:
--Create SCPS_APP Database to Store Application Files and Container Images
use role naspcs_role;
create database if not exists spcs_app;
create schema if not exists spcs_app.napp;
create stage if not exists spcs_app.napp.app_stage;
create image repository if not exists spcs_app.napp.img_repo;
create warehouse if not exists wh_nap with warehouse_size='xsmall';

#### Consumer Privilege Setup

In [ ]:
--Create NAC role and Grant Privileges
use role accountadmin;
create role if not exists nac;
grant role nac to role accountadmin;
create warehouse if not exists wh_nac with warehouse_size='xsmall';
grant usage on warehouse wh_nac to role nac with grant option;
grant imported privileges on database snowflake_sample_data to role nac;
grant create database on account to role nac;
grant bind service endpoint on account to role nac with grant option;
grant create compute pool on account to role nac;
grant create application on account to role nac;

In [ ]:
--Create Consumer Test Data Database
use role nac;
create database if not exists nac_test;
create schema if not exists nac_test.data;
use schema nac_test.data;
create view if not exists orders as select * from snowflake_sample_data.tpch_sf10.orders;

#### Build and Upload Container Images


In [ ]:
use role naspcs_role;
show image repositories in schema spcs_app.napp;

#### Create Application Package

In [ ]:
--Create Application Package and Grant Consumer Role Privileges
use role naspcs_role;
create application package spcs_app_pkg;
alter application package spcs_app_pkg add version v1 using @spcs_app.napp.app_stage;
grant install, develop on application package spcs_app_pkg to role nac;

#### Install & Run Application

In [ ]:
--Install App as the Consumer
use role nac;
create application spcs_app_instance from application package spcs_app_pkg using version v1;


In [ ]:
--Create Compute Pool and Grant Privileges
use database nac_test;
use role nac;
create  compute pool pool_nac for application spcs_app_instance
    min_nodes = 1 max_nodes = 1
    instance_family = cpu_x64_s
    auto_resume = true;

grant usage on compute pool pool_nac to application spcs_app_instance;
grant usage on warehouse wh_nac to application spcs_app_instance;
grant bind service endpoint on account to application spcs_app_instance;
CALL spcs_app_instance.v1.register_single_callback(
  'ORDERS_TABLE' , 'ADD', SYSTEM$REFERENCE('VIEW', 'NAC_TEST.DATA.ORDERS', 'PERSISTENT', 'SELECT'));

In [ ]:
--Start App Service
call spcs_app_instance.app_public.start_app('POOL_NAC', 'WH_NAC');

In [ ]:
--After running the above command you can run the following command to determine when the Service Endpoint is ready 
--Copy the endpoint, paste into a browser, and authenticate to the Snowflake account using the same credentials you've been using to log into Snowflake
call spcs_app_instance.app_public.app_url();

#### Clean up scripts

In [ ]:
--clean up consumer objects
use role nac;
drop application spcs_app_instance;
drop compute pool pool_nac;
drop database nac_test;

--clean up provider objects
use role naspcs_role;
drop application package spcs_app_pkg;
drop database spcs_app;
drop warehouse wh_nap;

--clean up prep objects
use role accountadmin;
drop warehouse wh_nac;
drop role naspcs_role;
drop role nac;